# **실습 9** 카페 Kiosk 자연어 인터페이스 (RAG)

**사용법:** 
환경설치(필요시) → 데이터 생성 → 인덱스 구축 → 로직 로딩 → 배치 리포트 실행

프론트엔드 없이, 질의 리스트를 입력하면 결과 리포트를 생성합니다.

## 1) 환경 설치

In [1]:
!pip -q install sentence-transformers faiss-cpu langchain langchain-openai python-dotenv

## 2) 메뉴 데이터 생성

*json

In [2]:

import json
from pathlib import Path

MENU = {
  "drinks": [
    {"id":"d_ame","name_kr":"아메리카노","name_en":"Americano","sizes":{"S":3500,"M":4000,"L":4500},"notes":"진한 에스프레소에 물을 더한 가장 기본 커피","pairing":["플레인 크루아상"],"tags":["coffee","americano","진함:중"]},
    {"id":"d_latte","name_kr":"카페 라떼","name_en":"Cafe Latte","sizes":{"S":4000,"M":4500,"L":5000},"notes":"에스프레소+우유 부드러운 밸런스","pairing":["치즈 케이크"],"tags":["coffee","latte","진함:하"]},
    {"id":"d_vanilla","name_kr":"바닐라 라떼","name_en":"Vanilla Latte","sizes":{"S":4500,"M":5000,"L":5500},"notes":"인기 메뉴, 바닐라 시럽의 달콤함","pairing":["마들렌","초코칩 쿠키"],"tags":["coffee","latte","sweet","진함:하"]},
    {"id":"d_capp","name_kr":"카푸치노","name_en":"Cappuccino","sizes":{"S":4000,"M":4500,"L":5000},"notes":"우유 거품이 풍부해 질감이 좋음","pairing":["아몬드 크루아상"],"tags":["coffee","foam","진함:중"]},
    {"id":"d_coldbrew","name_kr":"콜드 브루","name_en":"Cold Brew","sizes":{"S":4300,"M":4800,"L":5300},"notes":"산미 깔끔, 얼음과 함께 청량감","pairing":["플레인 크루아상"],"tags":["coffee","cold","acidity","진함:중상"]},
    {"id":"d_dolce","name_kr":"돌체 라떼","name_en":"Dolce Latte","sizes":{"S":4700,"M":5200,"L":5700},"notes":"연유 기반 달콤한 라떼","pairing":["티라미수"],"tags":["coffee","latte","very_sweet","진함:하"]},
    {"id":"d_espresso","name_kr":"에스프레소","name_en":"Espresso","sizes":{"S":3500},"notes":"가장 진한 커피 추출","pairing":[],"tags":["coffee","espresso","진함:상"]},
    {"id":"d_matcha","name_kr":"녹차 라떼","name_en":"Green Tea Latte","sizes":{"S":4300,"M":4800,"L":5300},"notes":"말차 풍미","pairing":["마카롱"],"tags":["noncoffee","latte","sweet"]},
    {"id":"d_choc","name_kr":"초콜릿 라떼","name_en":"Chocolate Latte","sizes":{"S":4300,"M":4800,"L":5300},"notes":"달콤 진한 초콜릿","pairing":["아몬드 크루아상","티라미수"],"tags":["noncoffee","sweet"]},
    {"id":"d_straw","name_kr":"딸기 스무디","name_en":"Strawberry Smoothie","sizes":{"M":5500,"L":6000},"notes":"과일 기반 블렌디드","pairing":["마카롱"],"tags":["noncoffee","fruit","sweet"]},
    {"id":"d_mango","name_kr":"망고 스무디","name_en":"Mango Smoothie","sizes":{"M":5500,"L":6000},"notes":"열대과일 상큼함","pairing":["마들렌"],"tags":["noncoffee","fruit","sweet"]}
  ],
  "desserts":[
    {"id":"ds_plain","name_kr":"플레인 크루아상","price":3000,"notes":"버터 풍미"},
    {"id":"ds_choc","name_kr":"초콜릿 크루아상","price":3500,"notes":"초콜릿 함유"},
    {"id":"ds_almond","name_kr":"아몬드 크루아상","price":3500,"notes":"아몬드 토핑"},
    {"id":"ds_madeleine","name_kr":"마들렌","price":2500,"notes":"버터향"},
    {"id":"ds_cookie","name_kr":"초콜릿 칩 쿠키","price":2800,"notes":"초코칩 가득"},
    {"id":"ds_macaron","name_kr":"마카롱","price":3000,"notes":"여러 맛"},
    {"id":"ds_cheese","name_kr":"치즈 케이크","price":6000,"notes":"라떼와 찰떡"},
    {"id":"ds_tiramisu","name_kr":"티라미수","price":6000,"notes":"커피 디저트"}
  ],
  "options":{
    "extra_shot":{"name_kr":"샷 추가","price":500,"applies_to":"coffee"},
    "syrup_vanilla":{"name_kr":"바닐라 시럽","price":300,"applies_to":"any"},
    "syrup_caramel":{"name_kr":"캐러멜 시럽","price":300,"applies_to":"any"},
    "syrup_hazelnut":{"name_kr":"헤이즐넛 시럽","price":300,"applies_to":"any"},
    "milk_lowfat":{"name_kr":"저지방 우유","price":0,"applies_to":"latte_family"},
    "milk_soy":{"name_kr":"두유","price":500,"applies_to":"latte_family"},
    "milk_almond":{"name_kr":"아몬드 브리즈","price":700,"applies_to":"latte_family"},
    "whipped":{"name_kr":"휘핑 크림","price":500,"applies_to":"choc_family"},
    "takeout_disc":{"name_kr":"테이크아웃 할인","price":-500,"applies_to":"any"}
  },
  "recommendations":[
    "바닐라 라떼는 가장 인기있는 메뉴입니다.",
    "치즈 케이크는 카페 라떼와 잘 어울립니다.",
    "따뜻한 아메리카노+플레인 크루아상 조합 추천",
    "마카롱은 달콤 음료와 잘 어울려요."
  ]
}

Path("menu.json").write_text(json.dumps(MENU, ensure_ascii=False, indent=2), encoding="utf-8")
print("menu.json saved")


menu.json saved


## 3) 메뉴 데이터 벡터화 및 검색 인덱스 구축

- 청킹
- 임베딩
- 인덱싱

In [3]:

import json, faiss, numpy as np
from sentence_transformers import SentenceTransformer
from pathlib import Path

def build_chunks(menu):
    chunks, meta = [], []
    for d in menu["drinks"]:
        sizes = ", ".join([f"{k}:{v}원" for k,v in d["sizes"].items()])
        chunks.append(f"[DRINK] {d['name_kr']} ({d['name_en']}) | sizes: {sizes} | notes: {d['notes']} | pairing: {', '.join(d['pairing'])} | tags:{','.join(d['tags'])}")
        meta.append({"type":"drink","id":d["id"],"name":d["name_kr"]})
    for ds in menu["desserts"]:
        chunks.append(f"[DESSERT] {ds['name_kr']} | price:{ds['price']}원 | notes:{ds['notes']}")
        meta.append({"type":"dessert","id":ds["id"],"name":ds["name_kr"]})
    for k,v in menu["options"].items():
        chunks.append(f"[OPTION] {v['name_kr']} | price:{v['price']}원 | applies_to:{v['applies_to']}")
        meta.append({"type":"option","id":k,"name":v["name_kr"]})
    chunks.append("[RECO] " + " | ".join(menu["recommendations"]))
    meta.append({"type":"recommendations"})
    return chunks, meta

menu = json.loads(Path("menu.json").read_text(encoding="utf-8"))
chunks, meta = build_chunks(menu)

emb_model = SentenceTransformer("intfloat/multilingual-e5-base")
embs = emb_model.encode(chunks, normalize_embeddings=True, convert_to_numpy=True)
index = faiss.IndexFlatIP(embs.shape[1])
index.add(embs)
faiss.write_index(index, "menu.faiss")
Path("menu_store.json").write_text(json.dumps({"meta":meta,"chunks":chunks}, ensure_ascii=False, indent=2), encoding="utf-8")
print("Index built:", len(chunks), "chunks, dim:", embs.shape[1])


Index built: 29 chunks, dim: 768


## 4) 챗봇 핵심 로직 구현 (인텐트 분류 + RAG + 응답 생성)

1. 의존성 및 초기 설정
- 라이브러리 임포트 및 데이터 로딩 (menu.json, FAISS 인덱스)
- multilingual-e5-base 임베딩 모델 로딩

2. 인텐트 분류
- **INTENT_PATTERNS**: 정규식 기반 의도 분류 (order, option, recommend, complex, menu_info)
- **route_intent()**: 사용자 질문을 의도별로 분류

3. 벡터 검색
- **retrieve()**: 질문 임베딩 → FAISS 검색 → 상위 4개 관련 문서 반환

4. 장바구니 관리
- **CartItem**: 주문 항목 데이터 클래스 (가격, 옵션, 수량)
- **OrderManager**: 장바구니 CRUD (추가, 요약, 옵션 적용)

5. 응답 생성
- **LLM 통합**: OpenAI API + 의도별 프롬프트 템플릿
- **규칙 기반 Fallback**: LLM 없이도 동작하는 키워드 기반 응답
- **answer()**: 통합 파이프라인 (분류 → 검색 → LLM/규칙 → 응답)

In [4]:
import re, json, os, faiss
from dataclasses import dataclass, field
from typing import List, Dict
from pathlib import Path
from sentence_transformers import SentenceTransformer

menu = json.loads(Path("menu.json").read_text(encoding="utf-8"))
store = json.loads(Path("menu_store.json").read_text(encoding="utf-8"))
index = faiss.read_index("menu.faiss")
emb_model = SentenceTransformer("intfloat/multilingual-e5-base")

INTENT_PATTERNS = {
  "order":     r"(주세요|주문|살게요|계산)",
  "option":    r"(추가|뺄|변경|우유|샷|시럽|휘핑|테이크아웃|사이즈 업|업사이즈|옵션)",
  "recommend": r"(추천|달콤|상큼|시원|인기|베스트|입문)",
  "complex":   r"(그리고|둘 다|함께|같이|비교|중에 뭐가|더|가격은 어떻게)",
  "menu_info": r"(가격|얼마|설명|성분|사이즈|크기|카페인|진해|연해|진함|산미)",
  "other":     r".*"
}
def route_intent(q:str)->str:
    # Explicit check for common option keywords like "우유" or "두유"
    if "우유" in q or "두유" in q or "변경" in q or "옵션" in q or "샷" in q or "시럽" in q or "휘핑" in q:
         return "option"

    # Handle "장바구니" separately
    if "장바구니" in q:
        return "cart_summary"

    # Check specific patterns
    for k,pat in INTENT_PATTERNS.items():
        if k != "other" and re.search(pat, q):
            return k

    # Default to 'other' if no specific pattern matches
    return "other"


def retrieve(q, topk=4):
    qv = emb_model.encode([q], normalize_embeddings=True, convert_to_numpy=True)
    D,I = index.search(qv, topk)
    return [{"text":store["chunks"][i], "meta":store["meta"][i], "score":float(D[0][j])} for j,i in enumerate(I[0])]

@dataclass
class CartItem:
    item_id:str; name:str; kind:str; size:str|None; base_price:int; qty:int=1
    options:List[Dict]=field(default_factory=list)
    def total_price(self): return (self.base_price + sum(o["price"] for o in self.options)) * self.qty

class OrderManager:
    def __init__(self, menu): self.menu=menu; self.items:List[CartItem]=[]
    def add_drink(self, name_kr:str, size:str="M"):
        d = next((x for x in self.menu["drinks"] if x["name_kr"]==name_kr), None)
        if not d: return "해당 음료를 찾지 못했어요."
        if size not in d["sizes"]: size = list(d["sizes"].keys())[0]
        self.items.append(CartItem(d["id"], d["name_kr"], "drink", size, d["sizes"][size]))
        return f"{d['name_kr']} {size} 추가했어요."
    def add_dessert(self, name_kr:str):
        ds = next((x for x in self.menu["desserts"] if x["name_kr"]==name_kr), None)
        if not ds: return "해당 디저트를 찾지 못했어요."
        self.items.append(CartItem(ds["id"], ds["name_kr"], "dessert", None, ds["price"]))
        return f"{ds['name_kr']} 추가했어요."
    def add_option(self, idx:int, option_key:str):
        if idx<0 or idx>=len(self.items): return "해당 항목 번호가 없어요."
        opt = self.menu["options"].get(option_key)
        if not opt: return "해당 옵션을 찾지 못했어요."
        self.items[idx].options.append({"key":option_key, "name":opt["name_kr"], "price":opt["price"]})
        return f"{self.items[idx].name}에 {opt['name_kr']} 적용(+{opt['price']}원)."
    def summary(self):
        lines=[]; total=0
        for i,it in enumerate(self.items):
            opt_txt = ", ".join([f"{o['name']}({o['price']}원)" for o in it.options]) if it.options else "옵션 없음"
            line = f"{i+1}) {it.name}{' '+it.size if it.size else ''} x{it.qty} - {it.base_price}원 | {opt_txt} => {it.total_price()}원"
            total += it.total_price(); lines.append(line)
        return "\n".join(lines) + (f"\n총액: {total}원" if lines else "현재 장바구니가 비어있어요.")



In [5]:
# Optional LLM
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY","")
try:
    if OPENAI_API_KEY:
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
        #llm = None
    else:
        llm = None
except Exception:
    llm = None

SYS_BASE = "당신은 카페 키오스크 AI입니다. 금액은 원 단위, 메뉴에 없는 정보는 추정하지 않습니다."
TPL_BASE = "[컨텍스트]\n{ctx}\n\n[사용자]\n{q}\n\n[규칙]\n- 모르는 것은 '메뉴에 없는 정보예요'라고 답함\n- 가격은 숫자+원\n- 각 메뉴는 'drink', 'dessert', 'option' 중 하나의 종류(type)를 가집니다. 사용자의 질문(예: '음료 추천')에 맞는 종류의 메뉴만 답변하세요."
#TPL_BASE = "[컨텍스트]\n{ctx}\n\n[사용자]\n{q}\n\n[규칙]\n- 모르는 것은 '메뉴에 없는 정보예요'라고 답함\n- 가격은 숫자+원\n- 필요 시 장바구니 보기를 제안"
templates = {
    "order":     "주문 질의입니다. 장바구니에 추가하고, 장바구니 상태를 요약해 주세요.\n"+TPL_BASE,
    "menu_info": "메뉴 정보 질의입니다. 항목의 가격/사이즈/간단 설명만 알려주세요.\n"+TPL_BASE,
    "recommend": "추천 질의입니다. 추천/페어링/달콤/진함 힌트를 반영해 1~2개 제안하고 근거 1줄.\n"+TPL_BASE,
    "option":    "옵션 질의입니다. 가능한 옵션과 추가금/적용대상을 기준으로 안내하세요.\n"+TPL_BASE,
    "complex":   "복합 질의입니다. 비교/동시주문 요청을 분해해 각 항목의 가격/특징을 1~2줄로 정리.\n"+TPL_BASE,
    "other":     "카페 업무와 무관한 질문입니다. 정중히 메뉴/주문 관련으로 유도하는 한 줄 응답만.\n"+TPL_BASE,
    "cart_summary": "장바구니 내용을 요약합니다.\n"+TPL_BASE
}
def run_llm(intent, ctx, q):
    if not llm:
        if intent == "cart_summary":
            return None
        return None

    prompt = f"{SYS_BASE}\n\n{templates[intent].format(ctx=ctx, q=q)}"
    try:
        return llm.invoke(prompt).content
    except Exception:
        if intent == "cart_summary":
            return None
        return None

In [6]:
# Offline fallback
def simple_answer(intent, q, ctx_docs, cart:OrderManager):
    q = q.strip()
    def find_drink_in_q():
        for d in menu["drinks"]:
            if d["name_kr"] in q: return d
        return None
    def find_two_drinks_in_q():
        found = [d for d in menu["drinks"] if d["name_kr"] in q]
        return found[:2]
    def size_in_q():
        if "라지" in q or " L" in q: return "L"
        if "스몰" in q or " S" in q: return "S"
        return "M"

    if intent=="cart_summary":
        return cart.summary()


    if intent=="order":
        d = find_drink_in_q()
        if d:
            size = "L" if ("라지" in q or " L" in q) else ("S" if ("스몰" in q or " S" in q) else "M")
            return cart.add_drink(d["name_kr"], size)
        for ds in menu["desserts"]:
            if ds["name_kr"] in q:
                return cart.add_dessert(ds["name_kr"])
        return "죄송해요, 어떤 메뉴를 주문하시겠어요?"


    if intent=="menu_info":
        d = find_drink_in_q()
        if d:
            sizes = ", ".join([f"{k}:{v}원" for k,v in d["sizes"].items()])
            return f"{d['name_kr']} 가격/사이즈: {sizes}. 설명: {d['notes']}"
        for ds in menu["desserts"]:
            if ds["name_kr"] in q:
                return f"{ds['name_kr']} 가격: {ds['price']}원. 설명: {ds['notes']}"
        return "원하시는 항목을 찾지 못했어요. 메뉴 이름을 함께 말씀해 주세요."

    if intent=="recommend":
        if "달콤" in q:
            return "바닐라 라떼(M 5000원) 또는 돌체 라떼(M 5200원) 추천드려요. 달콤한 풍미입니다."
        if "상큼" in q or "시원" in q:
            return "망고 스무디(5500원) 또는 딸기 스무디(5500원) 추천드려요. 과일 베이스로 상큼합니다."
        if "진해" in q or "진한" in q:
            return "에스프레소(3500원) 또는 콜드 브루(M 4800원) 추천드려요. 진한 풍미입니다."
        return "바닐라 라떼(M 5000원)가 인기 있고, 라떼는 치즈 케이크와 잘 어울려요."

    if intent=="option":
        msgs=[]
        if "샷" in q: msgs.append("샷 추가: +500원")
        if "두유" in q or "우유" in q: msgs.append("우유 변경: 저지방 0원 / 두유 +500원 / 아몬드브리즈 +700원 (라떼 계열)")
        if "시럽" in q: msgs.append("시럽: 바닐라/캐러멜/헤이즐넛 +300원")
        # Check for option keywords in the query to provide a general list if no specific option is mentioned
        if not msgs and any(word in q for word in ["옵션", "변경", "추가", "뺄", "사이즈"]):
             msgs.append("가능 옵션: 샷(+500), 시럽(+300), 우유변경(0~+700), 휘핑(+500), 테이크아웃(-500)")
        if not msgs: return "원하시는 옵션을 찾지 못했어요. 어떤 옵션을 말씀하시겠어요?"
        return " / ".join(msgs)


    if intent=="complex":
        pair = find_two_drinks_in_q()
        if len(pair)>=2:
            def strength(d):
                if "에스프레소" in d["name_kr"]: return 3
                if "콜드 브루" in d["name_kr"]: return 2
                if "아메리카노" in d["name_kr"]: return 2
                if "카푸치노" in d["name_kr"]: return 2
                return 1
            p1,p2 = pair[0], pair[1]
            s = "더 진한 편은 " + (p1["name_kr"] if strength(p1)>=strength(p2) else p2["name_kr"]) + " 입니다."
            p1sizes = ", ".join([f"{k}:{v}원" for k,v in p1["sizes"].items()])
            p2sizes = ", ".join([f"{k}:{v}원" for k,v in p2["sizes"].items()])
            return f"{p1['name_kr']}({p1sizes}) vs {p2['name_kr']}({p2sizes}). {s}"
        return "두 항목을 함께 말씀해 주시면 가격/특징을 비교해 드릴게요."

    if intent=="other":
        return "날씨 등은 안내하지 않지만, 메뉴 추천이나 주문을 도와드릴게요. 예: '바닐라 라떼 가격?'"

    # This should ideally not be reached if 'other' is the catch-all
    return "요청을 이해했어요. 메뉴명과 함께 다시 말씀해 주세요."

In [7]:
# orchestration
def answer(query, cart:OrderManager, show_ctx=False):
    intent = route_intent(query)
    ctx_docs = retrieve(query, topk=4)
    ctx = "\n".join([f"- {d['text']}" for d in ctx_docs])

    add_msg = None

    # The order processing logic is now primarily handled within the simple_answer for 'order' intent
    # Removed the redundant order processing outside of simple_answer

    resp = run_llm(intent, ctx, query)

    # Fallback to simple_answer if LLM is not available or fails
    if resp is None:
        resp = simple_answer(intent, query, ctx_docs, cart)

    # Append cart summary only if the intent is 'order', 'cart_summary', or if the query explicitly mentions '장바구니'
    extra = ""
    if intent == "order" or intent == "cart_summary" or "장바구니" in query:
         extra = "\n\n[장바구니]\n"+cart.summary()

    context_dbg = "\n\n[검색 컨텍스트 상위]\n"+ctx if show_ctx else ""
    action = f"\n(시스템) {add_msg}" if add_msg else "" # add_msg is always None now, can be removed if not needed elsewhere

    return intent, resp + extra + context_dbg # Removed action as add_msg is always None

## 5) 배치 리포트

*TA: 질의 리스트 입력 → 자동 결과 테이블 출력

In [ ]:
import pandas as pd
from IPython.display import display

user_queries = [
    "아메리카노 M 사이즈 가격이 얼마야?",
    "달콤한 음료 추천해줘.",
    "카페 라떼 하나 주문할게요.",
    "두유로 바꿔도 돼?",
    "아메리카노랑 카푸치노 중에 뭐가 더 진해요? 그리고 가격은 어떻게 되나요?",
    "치즈 케이크 가격은?",
    "옵션 뭐 있어?",
    "오늘 날씨 어때?",
    "장바구니 보여줘."
]
cart = OrderManager(menu)
rows=[]
for q in user_queries:
    intent, ans = answer(q, cart, show_ctx=False)
    rows.append({"query":q, "intent":intent, "answer":ans})

pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(rows)
display(df)

## 6) 시연 - 스크립트

In [ ]:
import pandas as pd

hti_cases = [
    {"id":"TS-001","task":"메뉴 탐색/정보","utter":"아메리카노 가격?","expect":"가격/사이즈/설명"},
    {"id":"TS-002","task":"추천(취향)","utter":"달콤한 음료 추천","expect":"1~2개+이유"},
    {"id":"TS-003","task":"옵션","utter":"라떼 두유로 변경 가능해?","expect":"가능/추가금"},
    {"id":"TS-004","task":"복합","utter":"아메리카노 vs 카푸치노 뭐가 진해? 가격은?","expect":"비교+가격"},
    {"id":"TS-005","task":"주문/장바구니","utter":"바닐라 라떼 하나 주세요","expect":"장바구니 반영"},
    {"id":"TS-006","task":"무관 질의","utter":"오늘 날씨 어때?","expect":"정중 리다이렉션"}
]

cart_demo = OrderManager(menu)

logs=[]

for c in hti_cases:
    intent, ans = answer(c["utter"], cart_demo, show_ctx=False)
    logs.append({"id":c["id"], "utter":c["utter"], "intent":intent, "resp":ans})

pd.set_option('display.max_colwidth', None)
df_demo = pd.DataFrame(logs)
display(df_demo)

print("\n--- Cart State after HTI Cases ---")
print(cart_demo.summary())

## 7) 시연 - 대화

In [ ]:
import os
from openai import OpenAI
client = OpenAI()

# ===== 기존 코드에서 제공된 객체/함수 가정 =====
# - menu: 메뉴 데이터
# - OrderManager(menu): 장바구니 및 주문 상태를 관리하는 클래스
# - route_intent(user_input): 인텐트 라우팅 함수
# - answer(user_input, cart, show_ctx=False): 규칙/DB 기반 사실 응답 반환 (intent, raw_answer)

cart = OrderManager(menu)

# ===== 대화 히스토리 및 요약 =====
messages = [
    {"role": "system", "content": "You are a polite Korean cafe AI assistant."}
]
conversation_summary = ""  # 길이 관리용 요약(간단 문자열)
MAX_TURNS = 12             # 유지할 최근 턴 수(유저-어시스턴트 쌍 기준)
def build_user_prompt(user_input: str, raw_answer: str, summary: str) -> str:
    base = [
        "당신은 친절한 카페 직원 AI입니다.",
        "아래의 '사실 데이터'를 바탕으로 자연스럽고 따뜻하게 대답하세요.",
        "가격·사이즈 등 숫자는 그대로 유지하세요.",
    ]
    if summary:
        base.append("\n[이전 대화 요약]\n" + summary.strip())
    base.append("\n[사용자 질문]\n" + user_input.strip())
    base.append("\n[사실 데이터]\n" + raw_answer.strip())
    base.append("\n[출력 형식]\n자연스러운 문장체(한국어), 필요한 경우 적당한 이모지 사용")
    return "\n".join(base)

def maybe_summarize_history(msgs, current_summary: str) -> str:
    user_lines = []
    for m in reversed(msgs):
        if m["role"] == "user":
            txt = m["content"].splitlines()[0]
            if len(txt) > 60:
                txt = txt[:60] + "..."
            user_lines.append(f"- {txt}")
        if len(user_lines) >= 6:
            break
    if not user_lines:
        return current_summary
    head = "최근 요청 요약:\n" + "\n".join(reversed(user_lines))
    if len(head) > 800:
        head = head[-800:]
    return head

turn_pairs = 0
chat_history = []  # ===> 콘솔 출력용 대화 기록

In [ ]:
# 챗봇 시연
print(":커피:️ 자연스러운 LLM 대화형 카페 키오스크 :커피:️")
print("그만두려면 quit 입력\n")

while True:
    user_input = input("User: ").strip()
    if user_input.lower() in ["quit", "exit"]:
        print("Chatbot: 오늘도 카페에 와주셔서 감사해요! :커피:️")
        break
    # 1) 인텐트 감지 및 사실 데이터 생성
    _intent = route_intent(user_input)
    _intent, raw_answer = answer(user_input, cart, show_ctx=False)
    
    # 2) 이전 대화 ‘요약 + 최근 메시지’와 함께 LLM 호출
    conversation_summary = maybe_summarize_history(messages, conversation_summary)
    llm_prompt = build_user_prompt(user_input, raw_answer, conversation_summary)
    
    # === 사용자 채팅 기록 표시 ===
    chat_history.append(f":상반신_그림자: User: {user_input}")
    
    # === 히스토리 누적 (LLM 입력용) ===
    messages.append({"role": "user", "content": llm_prompt})
    if len(messages) > 2 * MAX_TURNS + 1:
        keep = [messages[0]] + messages[-2 * MAX_TURNS:]
        messages = keep
    
    # === LLM 응답 ===
    llm_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=300,
        temperature=0.6,
    )
    
    final_answer = llm_response.choices[0].message.content.strip()

    # === 키오스크 응답 기록 표시 ===
    chat_history.append(f":로봇_얼굴: Kiosk: {final_answer}")
    print("\n" + "\n".join(chat_history[-4:]))  # 최근 2턴(유저+봇)만 화면에 갱신 표시
    print("-" * 60)
    
    # === 히스토리 누적 (LLM 내부용) ===
    messages.append({"role": "assistant", "content": final_answer})
    turn_pairs += 1
    if turn_pairs % 3 == 0:
        conversation_summary = maybe_summarize_history(messages, conversation_summary)